In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import pickle
import random

import torch
from torch import nn
from torch.utils.data import DataLoader

from fastai.vision.all import *

from datetime import datetime
from scipy.signal import savgol_filter
from scipy.signal import correlate
from sklearn.metrics import mean_squared_error
from IPython.core.magic import register_cell_magic

In [1]:
import tensorflow as tf
print(tf.__version__)

2024-09-25 14:17:52.549497: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-25 14:17:52.560615: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-25 14:17:52.572659: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-25 14:17:52.576362: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-25 14:17:52.585994: I tensorflow/core/platform/cpu_feature_guar

2.17.0


In [3]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

2.4.1+cu121
True
4
0


In [3]:
device = torch.device("cuda:0")
print(device)

cuda:0


In [7]:
x = torch.rand(1000000,256,device = device)
y = x.cuda()

In [8]:
print(x[0:5,0:5])
print(y.to("cpu", torch.double)[0:5,0:5])

tensor([[0.5838, 0.7997, 0.4081, 0.3213, 0.0968],
        [0.8383, 0.9666, 0.5852, 0.0980, 0.1693],
        [0.6443, 0.5052, 0.1315, 0.9768, 0.2860],
        [0.2568, 0.7425, 0.5485, 0.7974, 0.6843],
        [0.8687, 0.0254, 0.8877, 0.0741, 0.0439]], device='cuda:0')
tensor([[0.5838, 0.7997, 0.4081, 0.3213, 0.0968],
        [0.8383, 0.9666, 0.5852, 0.0980, 0.1693],
        [0.6443, 0.5052, 0.1315, 0.9768, 0.2860],
        [0.2568, 0.7425, 0.5485, 0.7974, 0.6843],
        [0.8687, 0.0254, 0.8877, 0.0741, 0.0439]], dtype=torch.float64)


In [22]:
from fastai.tabular.all import *
path = untar_data(URLs.ADULT_SAMPLE)

In [23]:
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
    cat_names = ['workclass', 'education', 'marital-status', 'occupation',
    'relationship', 'race'],
    cont_names = ['age', 'fnlwgt', 'education-num'],
    procs = [Categorify, FillMissing, Normalize])

learn = tabular_learner(dls, metrics=accuracy)

/home/sieglew/.local/lib/python3.9/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


In [25]:
learn.fit_one_cycle(12)

epoch,train_loss,valid_loss,accuracy,time
0,0.345538,0.357411,0.829545,00:04
1,0.359135,0.362926,0.829085,00:04
2,0.355792,0.359608,0.826628,00:04
3,0.354025,0.357982,0.830467,00:04
4,0.349482,0.368184,0.823249,00:04
5,0.343748,0.359266,0.828778,00:04
6,0.348321,0.362397,0.822789,00:04
7,0.354008,0.358884,0.827856,00:04
8,0.343268,0.354776,0.829392,00:04
9,0.341514,0.353782,0.830006,00:04


In [26]:
learn.show_results()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary,salary_pred
0,5.0,16.0,5.0,9.0,5.0,5.0,1.0,-1.142122,-0.266584,-0.024048,0.0,0.0
1,5.0,16.0,5.0,6.0,2.0,5.0,1.0,-1.142122,0.567496,-0.024048,0.0,0.0
2,5.0,3.0,5.0,9.0,4.0,5.0,1.0,-1.434863,-0.618067,-0.804646,0.0,0.0
3,5.0,13.0,3.0,11.0,1.0,5.0,1.0,1.199807,-1.131741,1.537147,1.0,1.0
4,7.0,8.0,1.0,5.0,2.0,5.0,1.0,0.102028,0.382803,0.756550,0.0,0.0
5,5.0,10.0,3.0,5.0,1.0,5.0,1.0,0.541140,-0.163912,1.146848,1.0,1.0
6,7.0,12.0,3.0,6.0,1.0,5.0,1.0,0.833881,-1.487889,-0.414347,0.0,0.0
7,5.0,12.0,3.0,15.0,1.0,2.0,1.0,-0.410269,-0.727973,-0.414347,0.0,0.0
8,3.0,13.0,7.0,11.0,5.0,5.0,1.0,0.760695,-1.048918,1.537147,0.0,0.0


In [32]:
def f(x): return x**2


In [37]:
xt = tensor(3.).requires_grad_()
f(xt).backward()

In [39]:
xt.grad

tensor(6.)

In [40]:
torch.arange(0,20).float()

tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
        14., 15., 16., 17., 18., 19.])

In [5]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')
tf.config.set_soft_device_placement(True)
tf.debugging.set_log_device_placement(True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]

In [16]:
x = torch.linspace(1,1000,1000)**2
y = x.cuda()

In [11]:

[(x,y,z) for x in range(1,1000) for y in range(x,1000) for z in range(y,1000) if x**2 + y**2 == z**2]

[(3, 4, 5),
 (5, 12, 13),
 (6, 8, 10),
 (7, 24, 25),
 (8, 15, 17),
 (9, 12, 15),
 (9, 40, 41),
 (10, 24, 26),
 (11, 60, 61),
 (12, 16, 20),
 (12, 35, 37),
 (13, 84, 85),
 (14, 48, 50),
 (15, 20, 25),
 (15, 36, 39),
 (15, 112, 113),
 (16, 30, 34),
 (16, 63, 65),
 (17, 144, 145),
 (18, 24, 30),
 (18, 80, 82),
 (19, 180, 181),
 (20, 21, 29),
 (20, 48, 52),
 (20, 99, 101),
 (21, 28, 35),
 (21, 72, 75),
 (21, 220, 221),
 (22, 120, 122),
 (23, 264, 265),
 (24, 32, 40),
 (24, 45, 51),
 (24, 70, 74),
 (24, 143, 145),
 (25, 60, 65),
 (25, 312, 313),
 (26, 168, 170),
 (27, 36, 45),
 (27, 120, 123),
 (27, 364, 365),
 (28, 45, 53),
 (28, 96, 100),
 (28, 195, 197),
 (29, 420, 421),
 (30, 40, 50),
 (30, 72, 78),
 (30, 224, 226),
 (31, 480, 481),
 (32, 60, 68),
 (32, 126, 130),
 (32, 255, 257),
 (33, 44, 55),
 (33, 56, 65),
 (33, 180, 183),
 (33, 544, 545),
 (34, 288, 290),
 (35, 84, 91),
 (35, 120, 125),
 (35, 612, 613),
 (36, 48, 60),
 (36, 77, 85),
 (36, 105, 111),
 (36, 160, 164),
 (36, 323, 325)

In [4]:
# Get cpu, gpu or mps device for training.
device = ("cuda" if torch.cuda.is_available() else "cpu")

# or select device manually
#device = torch.device("cuda:0")

print(f"Using {device} device")

Using cuda device


In [5]:
def quad_triples_1(dim):
    v = np.linspace(1,dim,dim)**2
    T = torch.from_numpy(np.array(np.transpose([v]*dim) + [v]*dim))
    idx = torch.unique(torch.sort(torch.where(torch.isin(T, torch.from_numpy(v)), T, torch.tensor([0])).nonzero() + torch.tensor([1]))[0], dim = 0)
    L1 = idx[:,0]**2
    L2 = idx[:,1]**2

    return list(map(tuple, np.transpose(torch.sqrt(torch.stack((L1, L2, L1+L2))).int().numpy())))

In [6]:
dim = 10000
triples = quad_triples_1(dim)

In [7]:
print(len(triples))

12471
